In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

# Sample text data
text = """Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice "without pictures or conversation?"
"""

# Create character mappings
chars = sorted(list(set(text)))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

# Prepare input and target data
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

# Define the RNN model
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars))),
    Dense(len(chars), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Function to sample the next character
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Train the model
model.fit(x, y, batch_size=128, epochs=20)

# Generate text
start_index = np.random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index : start_index + maxlen]
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ Temperature:', temperature)
    print(generated_text)
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_to_index[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = index_to_char[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        print(next_char, end='')
    print()

Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 3.3329
Epoch 2/20
1/1 [==============================] - 0s 54ms/step - loss: 3.3163
Epoch 3/20
1/1 [==============================] - 0s 50ms/step - loss: 3.2994
Epoch 4/20
1/1 [==============================] - 0s 46ms/step - loss: 3.2813
Epoch 5/20
1/1 [==============================] - 0s 52ms/step - loss: 3.2609
Epoch 6/20
1/1 [==============================] - 0s 90ms/step - loss: 3.2365
Epoch 7/20
1/1 [==============================] - 0s 73ms/step - loss: 3.2050
Epoch 8/20
1/1 [==============================] - 0s 95ms/step - loss: 3.1609
Epoch 9/20
1/1 [==============================] - 0s 86ms/step - loss: 3.0930
Epoch 10/20
1/1 [==============================] - 0s 50ms/step - loss: 2.9813
Epoch 11/20
1/1 [==============================] - 0s 47ms/step - loss: 2.8347
Epoch 12/20
1/1 [==============================] - 0s 48ms/step - loss: 2.9203
Epoch 13/20
1/1 [==============================] - 0s 49ms/step